In [1]:
# %matplotlib inline
import matplotlib
matplotlib.use('Agg')
import networkx as nx
from matplotlib import pyplot as plt
from itertools import permutations

In [2]:
g = nx.Graph()
for i in range(1, 4):
    g.add_edge(0, i)
g.add_edge(0, 4)
for i in range(5, 11):
    g.add_edge(4, i)
    

In [ ]:
pos = nx.spring_layout(g)

In [59]:
plt.clf()
nx.draw_networkx(g, pos=pos, node_size=1000, font_size=24, node_color='white')
plt.savefig('figs/example_network.pdf')

In [21]:
def build_random_walk_graph(g, infections):
    g = g.to_directed()
    for u, v in permutations(infections, 2):
        if g.has_edge(u, v) and infections[u] < infections[v]:
            g.remove_edge(u, v)
    return g

In [55]:
def query_importance_demo(g, pos, infections, alpha):
    plt.clf()
    colormap = {1: 'red',  # infected
                0: 'gray'}
    g = build_random_walk_graph(g, infections)

    infected_nodes = set(infections.keys())
    earliest_node = sorted(infections, key=lambda k: infections[k])[0]
    personalization = {n: 0 for n in  g.nodes_iter()}
    personalization[earliest_node] = 1
    print('personalization ={}'.format(personalization))
    pr = nx.pagerank(g, alpha=alpha,
                     personalization=personalization,
                     dangling=personalization,
                     nstart=personalization)
    print('node\' query importance:')
    for n, s in sorted(pr.items(), key=lambda t: t[1], reverse=True):
        # if n not in infections:
        print(n, s)        
    nx.draw_networkx_labels(g,
                            pos={k: v+0.09 for k, v in pos.items()},
                            labels={n: 't{}'.format(t) for n, t in infections.items()},
                            font_size=20)
    nx.draw_networkx(g.to_undirected(),
                     pos=pos,
                     font_size=20,
                     node_color=[colormap[int(n in infections)] for n in g.nodes_iter()],
                     node_size=[(pr[n]*12000+100 if n not in infections else 1000)
                                for n in g.nodes_iter()])

In [56]:
infections = {0: 0, 1: 1, 2: 1}  # 0 infected 1 and 2
query_importance_demo(g, pos, infections, alpha=0.85)
plt.savefig('figs/example_network_1.pdf')
# 4 should have maximal score
# then 3

personalization ={0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}
node' query importance:
4 0.3327714413133179
0 0.2980936817591244
3 0.12669035518872543
5 0.04040742028980534
6 0.04040742028980534
7 0.04040742028980534
8 0.04040742028980534
9 0.04040742028980534
10 0.04040742028980534
1 0.0
2 0.0


In [57]:
# 1 infected 0
infections = {1: 0, 0: 1}  # not reasonable
query_importance_demo(g, pos, infections, 0.85)
plt.savefig('figs/example_network_2.pdf')

# source is 1
# all other nodes have equal score

personalization ={0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}
node' query importance:
1 1.0
0 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0


In [58]:
# 0 infected 1 and 4
infections = {0: 0, 1: 1, 4: 1}
query_importance_demo(g, pos, infections, alpha=0.85)  # reasonable
plt.savefig('figs/example_network_3.pdf')

# 5 ... 8 should have 0
# 3 and 4 should be maximized

personalization ={0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0}
node' query importance:
0 0.5405382034979566
2 0.22973089825102172
3 0.22973089825102172
1 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
